# What are we doing?
Predict who survived (1) or died (0) based on input information.

Raw data is also availble on wikipedia.

In [226]:
import numpy as np
import pandas as pd
from random import shuffle, randint

from sklearn.cross_validation import train_test_split

Getting errors with sklearn? 

Scikit-learn requires:
<li>Python (>= 2.6 or >= 3.3),
<li>NumPy (>= 1.6.1),
<li>SciPy (>= 0.9).

Try:
conda install scikit-learn
OR
pip install -U scikit-learn

Still got probs?

python -m pip install --upgrade pip
pip install --user numpy scipy matplotlib ipython jupyter pandas sympy nose

In [227]:
#read in csv file
data = pd.read_csv("titanic_all.csv")

data.head() #look at first 5 rows of the data

,Unnamed: 0,row.names,pclass,survived,name,age,embarked,home.dest,room,ticket,boat,sex
0,0,1,1st,1,"Allen, Miss Elisabeth Walton",29.0000,Southampton,"St Louis, MO",B-5,24160 L221,2,female
1,1,2,1st,0,"Allison, Miss Helen Loraine",2.0000,Southampton,"Montreal, PQ / Chesterville, ON",C26,NaN,NaN,female
2,2,3,1st,0,"Allison, Mr Hudson Joshua Creighton",30.0000,Southampton,"Montreal, PQ / Chesterville, ON",C26,NaN,-135,male
3,3,4,1st,0,"Allison, Mrs Hudson J.C. (Bessie Waldo Daniels)",25.0000,Southampton,"Montreal, PQ / Chesterville, ON",C26,NaN,NaN,female
4,4,5,1st,1,"Allison, Master Hudson Trevor",0.9167,Southampton,"Montreal, PQ / Chesterville, ON",C22,NaN,11,male


# Useful functions
<li>date_frame.head()
<li>data.columns
<li>data_frame.shape


In [228]:
#We have a problem...look at the age column
meanAge = data['age'].mean()
meanAge = round(meanAge)
print("Mean age is {}".format(meanAge))

Mean age is 31


In [229]:
#We have a problem...look at the age column. Let's replace empty values with the average age.
data['age'].fillna(meanAge, inplace=True) #inplace means change the dataframe
#data['age'].head(20)

In [230]:
#Grab desired inputs to test one. Build a dataframe from them.
dataInputs= data[["pclass","age","sex"]]

dataInputs['age'] = dataInputs['age'].apply(np.int64)#ignore warning
#dataInputs.head(20)

C:\Users\Nick Kroeger\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [231]:
#Grab the expected outputs. (This is supervised learning)
expectedOutput = data[["survived"]]
#expectedOutput.head()

# Clean Data
Change pclass values from 1st -> 1, 2nd -> 2, 3rd -> 3. Then change male and female to binary.

In [232]:
#clean pclass
dataInputs["pclass"].replace("3rd", 3, inplace=True)
dataInputs["pclass"].replace("2nd", 2, inplace=True)
dataInputs["pclass"].replace("1st", 1, inplace=True)

C:\Users\Nick Kroeger\Anaconda3\lib\site-packages\pandas\core\generic.py:3117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [233]:
#Clean sex (sounds dirty, or does it??)
#Female -> 0, Male -> 1
dataInputs["sex"] = np.where(dataInputs["sex"] == "female", 0 ,1)
dataInputs.head()

C:\Users\Nick Kroeger\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,pclass,age,sex
0,1,29,0
1,1,2,0
2,1,30,1
3,1,25,0
4,1,0,1


# Breakdown the input data into test and train

In [234]:
#Hint: use train_test_split
inputTrain, inputTest, expectedOutputTrain, expectedOutputTest = train_test_split(dataInputs, expectedOutput, train_size = .7, random_state = 42)

#inputTest.head()
print(inputTrain.shape)
#inputTest.shape
print(expectedOutputTrain.shape)
print(inputTest.shape)
print(expectedOutputTest.shape)

(919, 3)
(919, 1)
(394, 3)
(394, 1)


# Get weight matrix

In [235]:
#Build our least squares classifier for 2 classes
D = inputTrain.shape[1] + 1 #num of attributes, +1 is for the intercept (column of 1s)
K = expectedOutputTrain.shape[1]

In [236]:
#look at powerpoint for these equations
sum1 = np.zeros((D,D))
sum2 = np.zeros((D,K))

In [237]:
expectedOutputTrain = np.asarray(expectedOutputTrain)
expectedOutputTest = np.asarray(expectedOutputTest)
inputTrain = np.asarray(inputTrain)
inputTest = np.asarray(inputTest)

In [238]:
lmbda = .001
i=0
sum1=0
sum2=0
for xi in inputTrain:
    xi = np.append(1, xi)
    yi=expectedOutputTrain[i][0]
    sum1 += np.dot(xi, xi.T) + lmbda   
    sum2 += xi*yi
    i += 1
print(sum1)
print(sum2)

1005909.919
[ 300  567 9201   95]


In [246]:
W=sum2/sum1
W = np.array(W)[np.newaxis]
W

array([[  2.98237441e-04,   5.63668763e-04,   9.14694231e-03,
          9.44418563e-05]])

In [247]:
total = expectedOutputTest.shape[0]
i=0
numCorrect=0
for i in range(total):
    x=inputTest[i]
    print(x)
    x = np.append(1,x)
    values = list(np.dot(W,x))     
    print(values)
    
    
    actual = list(expectedOutputTest[i].ravel())
    if y == actual:
        numCorrect+=1
accuracy=numCorrect/float(total)*100
print(accuracy)

[ 1 49  1]
[0.44915652134054779]
[0]
[ 2 31  1]
[0.28507522849070843]
[0]
[ 3 20  1]
[0.18502253182374354]
[0]
[ 2 48  1]
[0.44057324779197482]
[0]
[ 3 31  0]
[0.28554445539769552]
[0]
[ 1 43  0]
[0.39418042561323341]
[0]
[ 1 13  1]
[0.11986659811433663]
[0]
[3 6 1]
[0.056965339457994757]
[0]
[ 3 31  1]
[0.28563889725397468]
[0]
[ 1 27  1]
[0.24792379048008539]
[0]
[ 3 31  1]
[0.28563889725397468]
[0]
[ 2 31  1]
[0.28507522849070843]
[0]
[ 1 57  1]
[0.52233205983526132]
[0]
[ 3 31  1]
[0.28563889725397468]
[0]
[ 3 31  0]
[0.28554445539769552]
[0]
[ 3 43  1]
[0.39540220499604506]
[0]
[ 1 33  1]
[0.30280544435112061]
[0]
[ 1 32  1]
[0.2936585020392814]
[0]
[ 3 31  0]
[0.28554445539769552]
[0]
[ 2 28  1]
[0.25763440155519085]
[0]
[ 1 14  0]
[0.12891909856989664]
[0]
[ 3 36  0]
[0.33127916695689152]
[0]
[ 1 38  0]
[0.3484457140540374]
[0]
[ 2 18  0]
[0.16607053658051971]
[0]
[ 3 45  1]
[0.41369608961972348]
[0]
[ 3 31  1]
[0.28563889725397468]
[0]
[ 3 20  1]
[0.18502253182374354]
[0]
[ 2 1

array([1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0,
       1, 1,

In [14]:
#Compare side-by-side the predicted and actual values of survived.
predDF = pd.DataFrame(data=pred)
frames = [predDF, expectedOutputTest.reset_index().ix[:,1]]
modelTest = pd.concat(frames, axis=1)
modelTest.columns = ["Predicted", "Actual"]
modelTest

,Predicted,Actual
0,1,0
1,0,0
2,0,0
3,0,0
4,0,0
5,1,1
6,1,1
7,0,0
8,0,1
9,1,1


In [15]:
#What if we use SVM?
clf = svm.SVC(kernel='linear', C=1)
clf.fit(inputTrain, expectedOutputTrain)

C:\Users\Nick Kroeger\Anaconda3\lib\site-packages\sklearn\svm\base.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y_ = column_or_1d(y, warn=True)


SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [16]:
clf.score(inputTest, expectedOutputTest)

0.76267281105990781

NOW WHAT?
... you can use your own datasets and make predictions!